In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nex_nw500_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_nex_nw500.txt", header = None, delim_whitespace=True)
nex_nu500_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_nex_nu500.txt", header = None, delim_whitespace=True)
nex_nv500_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_nex_nv500.txt", header = None, delim_whitespace=True)
nex_ntpw_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_nex_ntpw.txt", header = None, delim_whitespace=True)
nex_nqv2m_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_nex_nqv2m.txt", header = None, delim_whitespace=True)


In [3]:
ex_nw500_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_ex_nw500.txt", header = None, delim_whitespace=True)
ex_nu500_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_ex_nu500.txt", header = None, delim_whitespace=True)
ex_nv500_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_ex_nv500.txt", header = None, delim_whitespace=True)
ex_ntpw_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_ex_ntpw.txt", header = None, delim_whitespace=True)
ex_nqv2m_7905 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_7905_ex_nqv2m.txt", header = None, delim_whitespace=True)


In [4]:
indicator_0614 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_0614_indicator.txt", header = None, delim_whitespace=True)
test_set_y_initial = indicator_0614[4]
test_set_y = test_set_y_initial.to_numpy().reshape(1, test_set_y_initial.shape[0])


In [5]:
nw500_0614 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_0614_nw500.txt", header = None, delim_whitespace=True)
nu500_0614 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_0614_nu500.txt", header = None, delim_whitespace=True)
nv500_0614 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_0614_nv500.txt", header = None, delim_whitespace=True)
ntpw_0614 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_0614_ntpw.txt", header = None, delim_whitespace=True)
nqv2m_0614 = pd.read_csv("/Users/MyFolders/MIT/Datasets/DJF_0614_nqv2m.txt", header = None, delim_whitespace=True)


In [6]:
def sigmoid(Z):   
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

def relu(Z):    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache

In [7]:
def relu_backward(dA, cache):   
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

def sigmoid_backward(dA, cache):    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [8]:
def initialize_parameters_deep(layer_dims):   
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1]) #*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [9]:
def linear_forward(A, W, b):
    
    Z = W.dot(A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [10]:
def linear_activation_forward(A_prev, W, b, activation):   
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    
    return A, cache


In [11]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    #print(AL)
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [12]:
def compute_cost(AL, Y):   
    m = Y.shape[1]

    # Compute loss from aL and y.
    cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [13]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1./m * np.dot(dZ,A_prev.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [14]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [15]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [16]:
def update_parameters(parameters, grads, learning_rate):   
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

In [122]:
def get_train_test(nex_names, ex_names, test_names):
    nex_combo_7905 = pd.concat(nex_names, axis = 1)
    nex_combo_7905["label"] = 0

    ex_combo_7905 = pd.concat(ex_names, axis = 1)
    ex_combo_7905["label"] = 1

    combo_7905_df = pd.concat([ex_combo_7905, nex_combo_7905])
    
    train_combo_y_initial = combo_7905_df['label'].to_numpy()
    train_combo_y = train_combo_y_initial.reshape(1, train_combo_y_initial.shape[0])

    train_combo_x = combo_7905_df.iloc[:,0:combo_7905_df.shape[1]-1].to_numpy().T

    test_df = pd.concat(test_names, axis = 1)

    test_combo_x = test_df.to_numpy().T
    
    return train_combo_x, test_combo_x, train_combo_y

In [18]:
# GRADED FUNCTION: L_layer_model

def L_layer_model(train_combo_x, train_combo_y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization. (≈ 1 line of code)
    ### START CODE HERE ###
    parameters = initialize_parameters_deep(layers_dims)
    ### END CODE HERE ###
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        ### START CODE HERE ### (≈ 1 line of code)
        AL, caches = L_model_forward(train_combo_x, parameters)
        ### END CODE HERE ###
        
        # Compute cost.
        ### START CODE HERE ### (≈ 1 line of code)
        cost = compute_cost(AL, train_combo_y)
        ### END CODE HERE ###
    
        # Backward propagation.
        ### START CODE HERE ### (≈ 1 line of code)
        grads = L_model_backward(AL, train_combo_y, caches)
        ### END CODE HERE ###
 
        # Update parameters.
        ### START CODE HERE ### (≈ 1 line of code)
        parameters = update_parameters(parameters, grads, learning_rate)
        ### END CODE HERE ###
                
        # Print the cost every 100 training example
#         if print_cost and i % 100 == 0:
#             print ("Cost after iteration %i: %f" %(i, cost))
#         if print_cost and i % 100 == 0:
#             costs.append(cost)
            
#     # plot the cost
#     plt.plot(np.squeeze(costs))
#     plt.ylabel('cost')
#     plt.xlabel('iterations (per hundreds)')
#     plt.title("Learning rate =" + str(learning_rate))
#     plt.show()
    
    return parameters

In [19]:
def conf_matrix(predictions, y):
    cm = np.zeros((2,2), dtype = int)
    for i in range(y.shape[1]):
        if(y[0,i] == 1 and predictions[0,i] == 1):
            cm[0,0] += 1
        if(y[0,i] == 1 and predictions[0,i] == 0):
            cm[0,1] += 1
        if(y[0,i] == 0 and predictions[0,i] == 1):
            cm[1,0] += 1
        if(y[0,i] == 0 and predictions[0,i] == 0):
            cm[1,1] += 1
#     cm[0,2] = np.sum(cm[0])
#     cm[1,2] = np.sum(cm[1])
#     cm[2,0] = np.sum(cm.T[0])
#     cm[2,1] = np.sum(cm.T[1])
#     cm[2,2] = np.sum(cm[2])
    #print(cm)
    return cm

In [20]:
def evalmetrics(cm):
    tp = cm[0,0]
    fn = cm[0,1]
    fp = cm[1,0]
    tn = cm[1,1]
    
    tpr = tp/(tp+fn)
    fpr = fp/(fp+tn)
    precision = tp/(tp+fp)
    f1 = 2*precision*tpr/(precision + tpr)
    
    return tpr, fpr, f1

In [21]:
def predict(X, y, parameters):
    
    m = X.shape[1]
    n = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((1,m), dtype = int)
    j = 0
    
    # Forward propagation
    probas, caches = L_model_forward(X, parameters)

    
    # convert probas to 0/1 predictions
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
#             print("Count is %i and day number is %i" %(j, i+1))
#             j = j+1
#             print(probas[0,i])
#             if(y[0,i] == 0):
#                 pred_extreme.append(i+1)
            p[0,i] = 1
        else:
#             print("Count is %i and day number is %i" %(j, i+1))
#             j = j+1
#             print(probas[0,i])
            p[0,i] = -1
    
    #print results
    #print ("predictions: " + str(p))
    #print ("true labels: " + str(y))
    #print("Accuracy: "  + str(np.sum((p == y)/m)))
        
    return p

# nw500, nqv2m

In [123]:
train_combo_x, test_combo_x, train_combo_y = get_train_test([nex_nw500_7905,nex_nqv2m_7905], [ex_nw500_7905, ex_nqv2m_7905], [nw500_0614, nqv2m_0614])
layer_dims = [train_combo_x.shape[0],2,2,2,1]
parameters = L_layer_model(train_combo_x, train_combo_y, layer_dims, learning_rate = 0.5, num_iterations = 170, print_cost=True)


In [124]:
predictions_test_1 = predict(test_combo_x, test_set_y, parameters)

In [125]:
predictions_train_1 = predict(train_combo_x, train_combo_y, parameters)

# nu500, nv500, nqv2m

In [126]:
train_combo_x, test_combo_x, train_combo_y = get_train_test([nex_nu500_7905, nex_nv500_7905, nex_nqv2m_7905], [ex_nu500_7905, ex_nv500_7905, ex_nqv2m_7905], [nu500_0614, nv500_0614, nqv2m_0614])
layer_dims = [train_combo_x.shape[0],3,4,3,1]
parameters = L_layer_model(train_combo_x, train_combo_y, layer_dims, learning_rate = 0.5, num_iterations = 180, print_cost=True)


In [127]:
predictions_test_2 = predict(test_combo_x, test_set_y, parameters)

In [128]:
predictions_train_2 = predict(train_combo_x, train_combo_y, parameters)

# nu500, nv500, nw500, nqv2m

In [129]:
train_combo_x, test_combo_x, train_combo_y = get_train_test([nex_nu500_7905, nex_nv500_7905,nex_nw500_7905, nex_nqv2m_7905], [ex_nu500_7905, ex_nv500_7905, ex_nw500_7905, ex_nqv2m_7905], [nu500_0614,nv500_0614,nw500_0614, nqv2m_0614])
layer_dims = [train_combo_x.shape[0],4,3,3,1]
parameters = L_layer_model(train_combo_x, train_combo_y, layer_dims, learning_rate = 1.2, num_iterations = 479, print_cost=True)   


In [130]:
predictions_test_3 = predict(test_combo_x, test_set_y, parameters)

In [131]:
predictions_train_3 = predict(train_combo_x, train_combo_y, parameters)

# nu500, nv500, nw500, ntpw

In [132]:
train_combo_x, test_combo_x, train_combo_y = get_train_test([nex_nu500_7905, nex_nv500_7905, nex_nw500_7905, nex_ntpw_7905], [ex_nu500_7905, ex_nv500_7905, ex_nw500_7905, ex_ntpw_7905], [nu500_0614, nv500_0614, nw500_0614, ntpw_0614])
layer_dims = [train_combo_x.shape[0],2,3,2,1]
parameters = L_layer_model(train_combo_x, train_combo_y, layer_dims, learning_rate = 0.5, num_iterations = 158, print_cost=True)


In [133]:
predictions_test_4 = predict(test_combo_x, test_set_y, parameters)

In [134]:
predictions_train_4 = predict(train_combo_x, train_combo_y, parameters)

# With the 3 special cases

In [135]:
majority_train = np.add(np.add(np.add(predictions_train_1, predictions_train_2), predictions_train_3), predictions_train_4)
# majority_train[majority_train > 0] = 1
# majority_train[majority_train <= 0] = 0

for i in range(majority_train.shape[1]):
    if majority_train[0,i] > 0:
        if (predictions_train_1[0,i] == 1) and (predictions_train_2[0,i] == 1) and (predictions_train_3[0,i] == 1) and  (predictions_train_4[0,i] == -1):
            majority_train[0,i] = 0
        elif (predictions_train_1[0,i] == 1) and (predictions_train_2[0,i] == 1) and (predictions_train_3[0,i] == -1) and  (predictions_train_4[0,i] == 1):
            majority_train[0,i] = 0
        else:
            majority_train[0,i] = 1
   
    else:
        if (predictions_train_1[0,i] == 1) and (predictions_train_2[0,i] == -1) and (predictions_train_3[0,i] == -1) and  (predictions_train_4[0,i] == 1):
            majority_train[0,i] = 1
        else:
            majority_train[0,i] = 0

In [136]:
cm_train = conf_matrix(majority_train, train_combo_y)
confusion_train = pd.DataFrame(cm_train, index=['actual_extreme (48)', 'actual_non-extreme (764)'],
                         columns=['predicted_extreme','predicted_non-extreme'])

print("Calibration statistics (1979 - 2005) :")
display(confusion_train)
tpr, fpr, f1 = evalmetrics(cm_train)
print("True positive rate: %f" %(tpr))
print("False positive rate: %f" %(fpr))
print("F1 score: %f" %(f1))

Calibration statistics (1979 - 2005) :


,predicted_extreme,predicted_non-extreme
actual_extreme (48),163,2
actual_non-extreme (764),42,2230


True positive rate: 0.987879
False positive rate: 0.018486
F1 score: 0.881081


In [137]:
# predictions_test_3[predictions_test_3 == -3] = -1
# predictions_test_4[predictions_test_4 == -3] = -1
majority = np.add(np.add(np.add(predictions_test_1, predictions_test_2), predictions_test_3), predictions_test_4)
# majority[majority > 0] = 1
# majority[majority <= 0] = 0

for i in range(majority.shape[1]):
    if majority[0,i] > 0:
        if (predictions_test_1[0,i] == 1) and (predictions_test_2[0,i] == 1) and (predictions_test_3[0,i] == 1) and  (predictions_test_4[0,i] == -1):
            majority[0,i] = 0
        elif (predictions_test_1[0,i] == 1) and (predictions_test_2[0,i] == 1) and (predictions_test_3[0,i] == -1) and  (predictions_test_4[0,i] == 1):
            majority[0,i] = 0
        else:
            majority[0,i] = 1
   
    else:
        if (predictions_test_1[0,i] == 1) and (predictions_test_2[0,i] == -1) and (predictions_test_3[0,i] == -1) and  (predictions_test_4[0,i] == 1):
            majority[0,i] = 1
        else:
            majority[0,i] = 0

In [138]:
cm_test = conf_matrix(majority, test_set_y)
confusion_test = pd.DataFrame(cm_test, index=['actual_extreme (48)', 'actual_non-extreme (764)'],
                         columns=['predicted_extreme','predicted_non-extreme'])

print("Validation statistics (2006 - 2014) :")
display(confusion_test)
tpr, fpr, f1 = evalmetrics(cm_test)
print("True positive rate: %f" %(tpr))
print("False positive rate: %f" %(fpr))
print("F1 score: %f" %(f1))

Validation statistics (2006 - 2014) :


,predicted_extreme,predicted_non-extreme
actual_extreme (48),31,17
actual_non-extreme (764),35,729


True positive rate: 0.645833
False positive rate: 0.045812
F1 score: 0.543860


In [139]:
fpdays = []
for i in range(test_set_y.shape[1]):
    if majority[0,i] == 1: 
        if test_set_y[0,i] == 0:
            fpdays.append(i)
            #print(i+1, predictions_test_1[0][i], predictions_test_2[0][i],predictions_test_3[0][i],predictions_test_4[0][i])

# +-1 day

In [140]:
truefp1 = len(fpdays)
j = 0
for i in fpdays:
    before = indicator_0614.iloc[i-1, 4]
    actual = indicator_0614.iloc[i], 4
    after = indicator_0614.iloc[i+1, 4]
    
    if (before == 1) or (after == 1):
        #print(fpdays[j]+1)
        truefp1 = truefp1 - 1
    j = j+1

In [141]:
cm_test[1,0] = truefp1
confusion_test = pd.DataFrame(cm_test, index=['actual_extreme (48)', 'actual_non-extreme (764)'],
                         columns=['predicted_extreme','predicted_non-extreme'])

print("Validation statistics (2006 - 2014) :")
display(confusion_test)
tpr, fpr, f1 = evalmetrics(cm_test)
print("True positive rate: %f" %(tpr))
print("False positive rate: %f" %(fpr))
print("F1 score: %f" %(f1))

Validation statistics (2006 - 2014) :


,predicted_extreme,predicted_non-extreme
actual_extreme (48),31,17
actual_non-extreme (764),21,729


True positive rate: 0.645833
False positive rate: 0.028000
F1 score: 0.620000


# +- 2 days

In [142]:
truefp2 = len(fpdays)
j = 0
for i in fpdays:
    
    before2 = indicator_0614.iloc[i-2, 4]
    before1 = indicator_0614.iloc[i-1, 4]
    actual = indicator_0614.iloc[i], 4
    after1 = indicator_0614.iloc[i+1, 4]
    after2 = indicator_0614.iloc[i+2, 4]
    
    if (before1 == 1) or (before2 == 1) or (after1 == 1) or (after2 == 1):
        #print(fpdays[j] + 1)
        truefp2 = truefp2 - 1
    j = j+1

#print(truefp2)

In [143]:
cm_test[1,0] = truefp2
confusion_test = pd.DataFrame(cm_test, index=['actual_extreme (48)', 'actual_non-extreme (764)'],
                         columns=['predicted_extreme','predicted_non-extreme'])

print("Validation statistics (2006 - 2014) :")
display(confusion_test)
tpr, fpr, f1 = evalmetrics(cm_test)
print("True positive rate: %f" %(tpr))
print("False positive rate: %f" %(fpr))
print("F1 score: %f" %(f1))

Validation statistics (2006 - 2014) :


,predicted_extreme,predicted_non-extreme
actual_extreme (48),31,17
actual_non-extreme (764),16,729


True positive rate: 0.645833
False positive rate: 0.021477
F1 score: 0.652632


# Without the 3 special cases

In [144]:
majority_train = np.add(np.add(np.add(predictions_train_1, predictions_train_2), predictions_train_3), predictions_train_4)
majority_train[majority_train > 0] = 1
majority_train[majority_train <= 0] = 0

In [145]:
majority = np.add(np.add(np.add(predictions_test_1, predictions_test_2), predictions_test_3), predictions_test_4)
majority[majority > 0] = 1
majority[majority <= 0] = 0

In [146]:
cm_train = conf_matrix(majority_train, train_combo_y)
confusion_train = pd.DataFrame(cm_train, index=['actual_extreme (48)', 'actual_non-extreme (764)'],
                         columns=['predicted_extreme','predicted_non-extreme'])

print("Calibration statistics (1979 - 2005) :")
display(confusion_train)
tpr, fpr, f1 = evalmetrics(cm_train)
print("True positive rate: %f" %(tpr))
print("False positive rate: %f" %(fpr))
print("F1 score: %f" %(f1))

Calibration statistics (1979 - 2005) :


,predicted_extreme,predicted_non-extreme
actual_extreme (48),165,0
actual_non-extreme (764),60,2212


True positive rate: 1.000000
False positive rate: 0.026408
F1 score: 0.846154


In [153]:
cm_test = conf_matrix(majority, test_set_y)
confusion_test = pd.DataFrame(cm_test, index=['actual_extreme (48)', 'actual_non-extreme (764)'],
                         columns=['predicted_extreme','predicted_non-extreme'])

print("Validation statistics (2006 - 2014) :")
display(confusion_test)
tpr, fpr, f1 = evalmetrics(cm_test)
print("True positive rate: %f" %(tpr))
print("False positive rate: %f" %(fpr))
print("F1 score: %f" %(f1))

Validation statistics (2006 - 2014) :


,predicted_extreme,predicted_non-extreme
actual_extreme (48),29,19
actual_non-extreme (764),41,723


True positive rate: 0.604167
False positive rate: 0.053665
F1 score: 0.491525


In [148]:
fpdays = []
for i in range(test_set_y.shape[1]):
    if majority[0,i] == 1: 
        if test_set_y[0,i] == 0:
            fpdays.append(i)
            #print(i+1, predictions_test_1[0][i], predictions_test_2[0][i],predictions_test_3[0][i],predictions_test_4[0][i])

# +- 1 day

In [149]:
truefp1 = len(fpdays)
j = 0
for i in fpdays:
    before = indicator_0614.iloc[i-1, 4]
    actual = indicator_0614.iloc[i], 4
    after = indicator_0614.iloc[i+1, 4]
    
    if (before == 1) or (after == 1):
        #print(fpdays[j]+1)
        truefp1 = truefp1 - 1
    j = j+1

In [154]:
cm_test[1,0] = truefp1
confusion_test = pd.DataFrame(cm_test, index=['actual_extreme (48)', 'actual_non-extreme (764)'],
                         columns=['predicted_extreme','predicted_non-extreme'])

print("Validation statistics (2006 - 2014) :")
display(confusion_test)
tpr, fpr, f1 = evalmetrics(cm_test)
print("True positive rate: %f" %(tpr))
print("False positive rate: %f" %(fpr))
print("F1 score: %f" %(f1))

Validation statistics (2006 - 2014) :


,predicted_extreme,predicted_non-extreme
actual_extreme (48),29,19
actual_non-extreme (764),24,723


True positive rate: 0.604167
False positive rate: 0.032129
F1 score: 0.574257


# +- 2 days

In [151]:
truefp2 = len(fpdays)
j = 0
for i in fpdays:
    
    before2 = indicator_0614.iloc[i-2, 4]
    before1 = indicator_0614.iloc[i-1, 4]
    actual = indicator_0614.iloc[i], 4
    after1 = indicator_0614.iloc[i+1, 4]
    after2 = indicator_0614.iloc[i+2, 4]
    
    if (before1 == 1) or (before2 == 1) or (after1 == 1) or (after2 == 1):
        #print(fpdays[j] + 1)
        truefp2 = truefp2 - 1
    j = j+1

#print(truefp2)

In [155]:
cm_test[1,0] = truefp2
confusion_test = pd.DataFrame(cm_test, index=['actual_extreme (48)', 'actual_non-extreme (764)'],
                         columns=['predicted_extreme','predicted_non-extreme'])

print("Validation statistics (2006 - 2014) :")
display(confusion_test)
tpr, fpr, f1 = evalmetrics(cm_test)
print("True positive rate: %f" %(tpr))
print("False positive rate: %f" %(fpr))
print("F1 score: %f" %(f1))

Validation statistics (2006 - 2014) :


,predicted_extreme,predicted_non-extreme
actual_extreme (48),29,19
actual_non-extreme (764),18,723


True positive rate: 0.604167
False positive rate: 0.024291
F1 score: 0.610526
